In [1]:
%cd searia_blog

/Users/damatta/searia_blog


In [3]:
import pandas as pd

data = {
    'Squad': ['Barcelona'],#, 'Palmeiras','Botafogo'], 
    'id': ['206d90db']}#, 'abdce579','d9fdd9d9']}

# data = {
#     'Squad': ['Botafogo'],#, 'Palmeiras','Botafogo'], 
#     'id': ['d9fdd9d9']}#, 'abdce579','d9fdd9d9']}

teams = pd.DataFrame(data)
teams.head()

,Squad,id
0,Barcelona,206d90db


In [13]:
import os
import time
import requests
import pandas as pd
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import csv
import re

#https://fbref.com/en/squads/639950ae/Flamengo-Stats
tables = ['matchlogs_for']
# years = ['2025']

def fetch_team_stats(table, id):
    url = f"https://fbref.com/en/squads/{id}"
    res = requests.get(url)
    res.raise_for_status()
    
    soup = BeautifulSoup(res.text, "html.parser")
    html = soup.find("table", id=table)
    #read into a df
    df = pd.read_html(str(html))[0]
    
    match_ids = []
    for row in html.tbody.find_all("tr"):
        link = row.find("a", string="Match Report")
        if link:
            m = re.search(r'/en/matches/([0-9a-f]+)/', link["href"])
            match_ids.append(m.group(1))
        else:
            match_ids.append(None)  # or '' if you prefer
            
    df["match_id"] = match_ids
    

    return df


for id in teams['id']:
    team = teams.loc[teams['id'] == id, 'Squad'].iloc[0]
    print(id, team)
    for table in tables:
#         for year in years:
            try:
                df = fetch_team_stats(table, id)
    #             print(df.head())
    #                print(f"{table}_{team}_{year}.csv")
                df.to_csv(f"{table}_{team}.csv", index=False)
            except Exception as e:
                print(f"No table found for: {table} in {id}")

206d90db Barcelona


In [14]:
import os
import time
import requests
import pandas as pd
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import csv
import re

year = 2025

# -------------------------------
# PoliteFetcher class definition
# -------------------------------
class PoliteFetcher:
    def __init__(self, base_url, user_agent, default_delay=8, cache_dir=None):
        self.base_url = base_url.rstrip('/')
        self.user_agent = user_agent
        self.default_delay = default_delay
        self.cache_dir = cache_dir
        self._init_robot_parser()

        if cache_dir:
            os.makedirs(cache_dir, exist_ok=True)

    def _init_robot_parser(self):
        robots_url = f"{self.base_url}/robots.txt"
        self.rp = RobotFileParser()
        self.rp.set_url(robots_url)
        self.rp.read()
        # Use crawl-delay if specified, else fallback
        self.delay = self.rp.crawl_delay('*') or self.default_delay

    def fetch(self, path, max_retries=10):
        """
        Fetches `base_url + path` if allowed. Honors crawl-delay,
        optionally caches pages, and retries on 429 with exponential backoff.
        """
        full_url = f"{self.base_url}{path}"
        # Check robots.txt permission
        if not self.rp.can_fetch('*', full_url):
            raise PermissionError(f"Blocked by robots.txt: {full_url}")

        # Prepare cache filename if caching enabled
        cache_path = None
        if self.cache_dir:
            safe_name = path.strip('/').replace('/', '_')
            cache_path = os.path.join(self.cache_dir, f"{safe_name}.html")
            if os.path.exists(cache_path):
                with open(cache_path, 'r', encoding='utf-8') as f:
                    return BeautifulSoup(f.read(), 'html.parser')

        headers = {'User-Agent': self.user_agent}
        wait = self.delay

        for attempt in range(1, max_retries + 1):
            resp = requests.get(full_url, headers=headers)
            if resp.status_code == 200:
                # Cache the page if required
                if cache_path:
                    with open(cache_path, 'w', encoding='utf-8') as f:
                        f.write(resp.text)
                time.sleep(self.delay)
                return BeautifulSoup(resp.text, 'html.parser')
            elif resp.status_code == 429:
                print(f"[429] Rate limited. Backing off for {wait}s (attempt {attempt})")
                time.sleep(wait)
                wait *= 2
            else:
                resp.raise_for_status()

        raise RuntimeError(f"Failed to fetch {full_url} after {max_retries} retries")

# ---------------------------------------
# Initialization and configuration block
# ---------------------------------------
# Assumes 'teams' (DataFrame with 'Squad') and 'year' are defined
# Example:
# teams = pd.read_csv("teams.csv")
# year = 2025

fetcher = PoliteFetcher(
    base_url="https://fbref.com",
    user_agent="MyAnalyticsBot/1.0 (+https://yourdomain.com/contact)",
    default_delay=8,
    cache_dir="fbref_cache"
)

tables = [
    'stats_206d90db_summary', 'stats_206d90db_passing', 'stats_206d90db_passing_types',
    'stats_206d90db_defense', 'stats_206d90db_possession', 'stats_206d90db_misc'
]

# ---------------------------------------
# Main scraping loop
# ---------------------------------------
squads = teams['Squad'].to_numpy()

for squad in squads:
    # Load match IDs for this squad
    match_logs_file = f"matchlogs_for_{squad}.csv"
    match_ids_df = pd.read_csv(match_logs_file)
    match_ids = match_ids_df['match_id'].dropna().astype(str).tolist()
#     match_ids = ['471380e2']

    for match_id in match_ids:
        print(f"Processing match {match_id}")
        for table in tables:
            try:
                # Fetch and parse the page
                soup = fetcher.fetch(f"/en/matches/{match_id}/")
                html_table = soup.find("table", id=table)
                if html_table is None:
                    print(f"  Table {table} not found for match {match_id}")
                    continue

                # Read table into DataFrame
                df = pd.read_html(str(html_table))[0]
                # Add match_id column
                df["match_id"] = match_id
                df['match_id'] = df['match_id'].astype(str)
                # Clean column names
                df.columns = [
                    col[1] if col[0] == "Unnamed: 0" else col[1] or col[0]
                    for col in df.columns.values
                ]
                # Save to CSV
                out_file = f"{table}_{match_id}.csv"
                df.to_csv(out_file, index=False, quoting=csv.QUOTE_NONNUMERIC)
                print(f"  Saved {out_file}")
            except PermissionError as e:
                print(f"  PermissionError: {e}")
            except Exception as e:
                print(f"  Error fetching {table} for match {match_id}: {e}")


Processing match 6f0fac43
  Saved stats_206d90db_summary_6f0fac43.csv
  Saved stats_206d90db_passing_6f0fac43.csv
  Saved stats_206d90db_passing_types_6f0fac43.csv
  Saved stats_206d90db_defense_6f0fac43.csv
  Saved stats_206d90db_possession_6f0fac43.csv
  Saved stats_206d90db_misc_6f0fac43.csv
Processing match 87bc9bba
  Saved stats_206d90db_summary_87bc9bba.csv
  Saved stats_206d90db_passing_87bc9bba.csv
  Saved stats_206d90db_passing_types_87bc9bba.csv
  Saved stats_206d90db_defense_87bc9bba.csv
  Saved stats_206d90db_possession_87bc9bba.csv
  Saved stats_206d90db_misc_87bc9bba.csv
Processing match f8fc00c7
  Saved stats_206d90db_summary_f8fc00c7.csv
  Saved stats_206d90db_passing_f8fc00c7.csv
  Saved stats_206d90db_passing_types_f8fc00c7.csv
  Saved stats_206d90db_defense_f8fc00c7.csv
  Saved stats_206d90db_possession_f8fc00c7.csv
  Saved stats_206d90db_misc_f8fc00c7.csv
Processing match 3ed1b8ff
  Saved stats_206d90db_summary_3ed1b8ff.csv
  Saved stats_206d90db_passing_3ed1b8ff.cs

  Table stats_206d90db_defense not found for match 0c126206
  Table stats_206d90db_possession not found for match 0c126206
  Table stats_206d90db_misc not found for match 0c126206
Processing match 1fcb527f
  Saved stats_206d90db_summary_1fcb527f.csv
  Table stats_206d90db_passing not found for match 1fcb527f
  Table stats_206d90db_passing_types not found for match 1fcb527f
  Table stats_206d90db_defense not found for match 1fcb527f
  Table stats_206d90db_possession not found for match 1fcb527f
  Table stats_206d90db_misc not found for match 1fcb527f
Processing match ac3f8e9a
  Saved stats_206d90db_summary_ac3f8e9a.csv
  Saved stats_206d90db_passing_ac3f8e9a.csv
  Saved stats_206d90db_passing_types_ac3f8e9a.csv
  Saved stats_206d90db_defense_ac3f8e9a.csv
  Saved stats_206d90db_possession_ac3f8e9a.csv
  Saved stats_206d90db_misc_ac3f8e9a.csv
Processing match b72a4588
  Saved stats_206d90db_summary_b72a4588.csv
  Saved stats_206d90db_passing_b72a4588.csv
  Saved stats_206d90db_passing_typ

  Saved stats_206d90db_misc_4b3ea62f.csv
Processing match af94853a
  Saved stats_206d90db_summary_af94853a.csv
  Saved stats_206d90db_passing_af94853a.csv
  Saved stats_206d90db_passing_types_af94853a.csv
  Saved stats_206d90db_defense_af94853a.csv
  Saved stats_206d90db_possession_af94853a.csv
  Saved stats_206d90db_misc_af94853a.csv
Processing match ff6aaaa9
  Saved stats_206d90db_summary_ff6aaaa9.csv
  Saved stats_206d90db_passing_ff6aaaa9.csv
  Saved stats_206d90db_passing_types_ff6aaaa9.csv
  Saved stats_206d90db_defense_ff6aaaa9.csv
  Saved stats_206d90db_possession_ff6aaaa9.csv
  Saved stats_206d90db_misc_ff6aaaa9.csv
Processing match f1804d9c
  Saved stats_206d90db_summary_f1804d9c.csv
  Saved stats_206d90db_passing_f1804d9c.csv
  Saved stats_206d90db_passing_types_f1804d9c.csv
  Saved stats_206d90db_defense_f1804d9c.csv
  Saved stats_206d90db_possession_f1804d9c.csv
  Saved stats_206d90db_misc_f1804d9c.csv
Processing match 45f486b1
  Saved stats_206d90db_summary_45f486b1.csv
 

In [15]:
import os
import glob

# Pattern for your files
pattern = "stats_206d90db_passing_types_*.csv"

for old_path in glob.glob(pattern):
    # e.g. old_path = "stats_639950ae_passing_types_03c34ee9.csv"
    
    # Decide on your new prefix (here we drop the leading “stats_639950ae_”)
    new_filename = old_path.replace("stats_206d90db_passing_types_", "stats_206d90db_pass_types_")
    
    # Perform the rename
    os.rename(old_path, new_filename)
    print(f"Renamed: {old_path} → {new_filename}")

Renamed: stats_206d90db_passing_types_231f83bc.csv → stats_206d90db_pass_types_231f83bc.csv
Renamed: stats_206d90db_passing_types_b299fca9.csv → stats_206d90db_pass_types_b299fca9.csv
Renamed: stats_206d90db_passing_types_e6bc2844.csv → stats_206d90db_pass_types_e6bc2844.csv
Renamed: stats_206d90db_passing_types_41097255.csv → stats_206d90db_pass_types_41097255.csv
Renamed: stats_206d90db_passing_types_f8fc00c7.csv → stats_206d90db_pass_types_f8fc00c7.csv
Renamed: stats_206d90db_passing_types_af94853a.csv → stats_206d90db_pass_types_af94853a.csv
Renamed: stats_206d90db_passing_types_3ed1b8ff.csv → stats_206d90db_pass_types_3ed1b8ff.csv
Renamed: stats_206d90db_passing_types_6b455e82.csv → stats_206d90db_pass_types_6b455e82.csv
Renamed: stats_206d90db_passing_types_6e84530a.csv → stats_206d90db_pass_types_6e84530a.csv
Renamed: stats_206d90db_passing_types_86447b37.csv → stats_206d90db_pass_types_86447b37.csv
Renamed: stats_206d90db_passing_types_f1804d9c.csv → stats_206d90db_pass_types_f

In [16]:
import glob
import pandas as pd

barcelona_2025 = pd.read_csv("matchlogs_for_Barcelona.csv")

#### Summary ####
# 1. Use glob to find all matching files in the current directory
files = glob.glob("stats_206d90db_summary_*.csv")

# 2. Read each CSV into a DataFrame
df_list = [pd.read_csv(f, dtype={'match_id': str}) for f in files]

# 3. Concatenate them into one big DataFrame
summary = pd.concat(df_list, ignore_index=True)

# 4. Optional: inspect
print(f"Read {len(files)} files, combined shape = {summary.shape}")

#### Defense ####
# 1. Use glob to find all matching files in the current directory
files = glob.glob("stats_206d90db_defense_*.csv")

# 2. Read each CSV into a DataFrame
df_list = [pd.read_csv(f, dtype={'match_id': str}) for f in files]

# 3. Concatenate them into one big DataFrame
defense = pd.concat(df_list, ignore_index=True)

# 4. Optional: inspect
print(f"Read {len(files)} files, combined shape = {defense.shape}")

#### Misc ####
# 1. Use glob to find all matching files in the current directory
files = glob.glob("stats_206d90db_misc_*.csv")

# 2. Read each CSV into a DataFrame
df_list = [pd.read_csv(f, dtype={'match_id': str}) for f in files]

# 3. Concatenate them into one big DataFrame
misc = pd.concat(df_list, ignore_index=True)

# 4. Optional: inspect
print(f"Read {len(files)} files, combined shape = {misc.shape}")

#### Passing ####
# 1. Use glob to find all matching files in the current directory
files = glob.glob("stats_206d90db_passing_*.csv")

# 2. Read each CSV into a DataFrame
df_list = [pd.read_csv(f, dtype={'match_id': str}) for f in files]

# 3. Concatenate them into one big DataFrame
passing = pd.concat(df_list, ignore_index=True)

# 4. Optional: inspect
print(f"Read {len(files)} files, combined shape = {passing.shape}")

#### Passing Types ####
# 1. Use glob to find all matching files in the current directory
files = glob.glob("stats_206d90db_pass_types_*.csv")

# 2. Read each CSV into a DataFrame
df_list = [pd.read_csv(f, dtype={'match_id': str}) for f in files]

# 3. Concatenate them into one big DataFrame
pass_types = pd.concat(df_list, ignore_index=True)

# 4. Optional: inspect
print(f"Read {len(files)} files, combined shape = {pass_types.shape}")

#### Possession ####
# 1. Use glob to find all matching files in the current directory
files = glob.glob("stats_206d90db_possession_*.csv")

# 2. Read each CSV into a DataFrame
df_list = [pd.read_csv(f, dtype={'match_id': str}) for f in files]

# 3. Concatenate them into one big DataFrame
possession = pd.concat(df_list, ignore_index=True)

# 4. Optional: inspect
print(f"Read {len(files)} files, combined shape = {possession.shape}")

Read 58 files, combined shape = (971, 40)
Read 50 files, combined shape = (838, 23)
Read 50 files, combined shape = (838, 23)
Read 50 files, combined shape = (838, 29)
Read 50 files, combined shape = (838, 22)
Read 50 files, combined shape = (838, 29)


In [72]:
import sqlite3
import pandas as pd
from IPython.display import HTML

# create an in-memory SQLite DB
conn = sqlite3.connect(':memory:')

barcelona_2025.to_sql('barcelona_2025', conn, index=False, if_exists='replace')

summary.to_sql('summary', conn, index=False, if_exists='replace')
defense.to_sql('defense', conn, index=False, if_exists='replace')
passing.to_sql('passing', conn, index=False, if_exists='replace')
pass_types.to_sql('pass_types', conn, index=False, if_exists='replace')
misc.to_sql('misc', conn, index=False, if_exists='replace')
possession.to_sql('possession', conn, index=False, if_exists='replace')

# now use pandas’ read_sql_query ### Raphinha | Lamine Yamal
query = """
        SELECT date, Comp, Opponent, Min, SoT
        FROM summary
        LEFT JOIN (
             SELECT match_id, CAST(REPLACE(Round, 'Matchweek ', '') as Int) as week, DATE(Date) as date, Comp, Opponent
             FROM barcelona_2025 
             WHERE Comp IN ('La Liga', 'Champions Lg')
             --AND CAST(REPLACE(Round, 'Matchweek ', '') as Int) <= 18
         ) as matches
         ON summary.match_id = matches.match_id
        WHERE Player = 'Lamine Yamal'
        --AND week <= 18
        --AND date > '2024-12-22'
        AND date IS NOT NULL
        ORDER BY date
         """

result = pd.read_sql_query(query, conn)
HTML(result.to_html(notebook=True))

,date,Comp,Opponent,Min,SoT
0,2024-08-17,La Liga,Valencia,85,1
1,2024-08-24,La Liga,Athletic Club,89,1
2,2024-08-27,La Liga,Rayo Vallecano,89,1
3,2024-08-31,La Liga,Valladolid,90,0
4,2024-09-15,La Liga,Girona,89,4
5,2024-09-19,Champions Lg,fr Monaco,79,1
6,2024-09-22,La Liga,Villarreal,90,0
7,2024-09-25,La Liga,Getafe,90,1
8,2024-09-28,La Liga,Osasuna,32,1
9,2024-10-01,Champions Lg,ch Young Boys,74,2
